In [37]:
import torch
from torch import autograd
from torch.distributions import Normal, MultivariateNormal, Independent
from torch.distributions.kl import kl_divergence

In [10]:
ones = torch.ones(3)
torch.diag(ones)

tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]])

In [11]:
loc = torch.zeros(3)
scale = torch.ones(3)
print(loc)
print(scale)
print(torch.diag(scale))

tensor([0., 0., 0.])
tensor([1., 1., 1.])
tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]])


## Experimenting with Independent

Independent "Reinterprets some of the batch dims of a distribution as event dims."

In [14]:
mvn = MultivariateNormal(loc, scale_tril=torch.diag(scale))

In [18]:
[mvn.batch_shape, mvn.event_shape]

[torch.Size([]), torch.Size([3])]

In [19]:
mvn.sample()

tensor([-0.3483, -0.8625, -0.6323])

In [20]:
normal = Normal(loc, scale)

In [21]:
[normal.batch_shape, normal.event_shape]

[torch.Size([3]), torch.Size([])]

In [25]:
normal.sample()

tensor([-0.5116, -0.3467,  0.3212])

```
>>> loc = torch.zeros(3)
>>> scale = torch.ones(3)
>>> mvn = MultivariateNormal(loc, scale_tril=torch.diag(scale))
>>> [mvn.batch_shape, mvn.event_shape]
[torch.Size(()), torch.Size((3,))]
>>> normal = Normal(loc, scale)
>>> [normal.batch_shape, normal.event_shape]
[torch.Size((3,)), torch.Size(())]
>>> diagn = Independent(normal, 1)
>>> [diagn.batch_shape, diagn.event_shape]
[torch.Size(()), torch.Size((3,))]
```

In [33]:
diagn = Independent(normal, 1)
[diagn.batch_shape, diagn.event_shape]

[torch.Size([]), torch.Size([3])]

In [34]:
diagn.sample()

tensor([ 1.9519, -0.9673,  0.1015])

"This is mainly useful for changing the shape of the result of log_prob."

In [35]:
normal.log_prob(normal.sample())

tensor([-0.9236, -1.2489, -1.6215])

In [36]:
diagn.log_prob(diagn.sample())

tensor(-3.1684)

## Autograd

```
RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn
```

In [80]:
x = torch.tensor([1.,2.,3.], requires_grad=True)
x

tensor([1., 2., 3.], requires_grad=True)

```
RuntimeError: grad can be implicitly created only for scalar outputs
```

In [81]:
def f(x):
    return (x**2).sum()

```
df(x)/dx =
[df(x1)/dx1, df(x2)/dx2, df(x3)/dx3] =
[2*x1, 2*x2, 2*x3] =
[2, 4, 6] =
```

In [82]:
autograd.grad(f(x), x)

(tensor([2., 4., 6.]),)

## Dotdict

In [ ]:
class dotdict(dict):
    '''A dict with dot access and autocompletion.
    
    The idea and most of the code was taken from 
    http://stackoverflow.com/a/23689767,
    http://code.activestate.com/recipes/52308-the-simple-but-handy-collector-of-a-bunch-of-named/
    http://stackoverflow.com/questions/2390827/how-to-properly-subclass-dict-and-override-get-set
    '''
    
    def __init__(self,*a,**kw):
        dict.__init__(self)
        self.update(*a, **kw)
        self.__dict__ = self
    
    def __setattr__(self, key, value):
        if key in dict.__dict__:
            raise AttributeError('This key is reserved for the dict methods.')
        dict.__setattr__(self, key, value)
    
    def __setitem__(self, key, value):
        if key in dict.__dict__:
            raise AttributeError('This key is reserved for the dict methods.')
        dict.__setitem__(self, key, value)
        
    def update(self, *args, **kwargs):
        for k, v in dict(*args, **kwargs).iteritems():
            self[k] = v
        
    def __getstate__(self):
        return self
 
    def __setstate__(self, state):
        self.update(state)
        self.__dict__ = self

In [ ]:
d = dotdict({'one':1})

In [3]:
class dotdict(dict):
    '''A dict with dot access and autocompletion.
    
    The idea and most of the code was taken from 
    http://stackoverflow.com/a/23689767,
    http://code.activestate.com/recipes/52308-the-simple-but-handy-collector-of-a-bunch-of-named/
    http://stackoverflow.com/questions/2390827/how-to-properly-subclass-dict-and-override-get-set
    '''
    
    def __init__(self,*a,**kw):
        dict.__init__(self)
        self.update(*a, **kw)
        self.__dict__ = self
    
    def __setattr__(self, key, value):
        if key in dict.__dict__:
            raise AttributeError('This key is reserved for the dict methods.')
        dict.__setattr__(self, key, value)
    
    def __setitem__(self, key, value):
        if key in dict.__dict__:
            raise AttributeError('This key is reserved for the dict methods.')
        dict.__setitem__(self, key, value)
        
    def update(self, *args, **kwargs):
        for k, v in dict(*args, **kwargs).iteritems():
            self[k] = v
        
    def __getstate__(self):
        return self
 
    def __setstate__(self, state):
        self.update(state)
        self.__dict__ = self

In [4]:
d = dotdict({'one':1})

AttributeError: 'dict' object has no attribute 'iteritems'